# Introduction
In this guide, we'll implement **linear regression** from scratch using **gradient descent**. Starting with **dataset loading**, we'll cover the **mathematical foundations** and step-by-step code implementation.

The goal is to understand how **linear regression** works, how **gradient descent** optimizes model parameters, and how to build it without **high-level machine learning libraries**.


### Table of Contents

1. **Importing Libraries**  
   Setting up the necessary libraries for data manipulation, model implementation, and visualization.

2. **Loading and Exploring the Dataset**  
   Understanding the structure of the dataset and initial data exploration.

3. **Preparing the Data**  
   Preprocessing the data by scaling features and splitting into training and testing sets.

4. **Initializing Parameters**  
   Defining the initial parameters for the model, including weights and bias.

5. **Defining the Prediction Function**  
   Implementing the model's prediction function to make estimates based on input data.

6. **Defining the Cost Function**  
   Formulating the cost function to measure the accuracy of predictions against actual values.

7. **Computing the Gradients**  
   Calculating the gradients for weights and bias to optimize the cost function.

8. **Updating Parameters Using Gradient Descent**  
   Applying gradient descent to adjust parameters and minimize the cost function.

9. **Training the Model**  
   Training the model using the data and updating parameters through iterative optimization.

10. **Evaluating Model Performance with Test Data**  
    Assessing the model's performance using test data and relevant metrics.

11. **Conclusion**  
    Summarizing the key findings and insights from the model implementation.

12. **Comparison with Sklearn Linear Regression**  
    Side by side comparison of the algorithm that we've written with the algorithms predefined in sklearn to check performance


# 1. Importing Libraries

The following code imports essential libraries for linear regression and dataset loading:

- **numpy**: For numerical computing and array manipulation.
- **load_diabetes**: Loads the Diabetes dataset for regression tasks.
- **matplotlib.pyplot**: For visualizations such as loss curves and predictions.

In [32]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes

# 2. Loading and Exploring the Dataset

This code loads the **Diabetes** dataset and prints:

- **X**: Feature matrix (442 samples, 10 features).
- **y**: Target vector (442 values, diabetes progression).

It also displays the feature names, the first five samples of X, and the first five target values.


In [33]:
data = load_diabetes()

X = data.data         # Feature matrix (shape: [442, 10])
y = data.target       # Target vector (shape: [442,])

print('Feature names:', data.feature_names)
print('First five target values:', y[:5])

Feature names: ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
First five target values: [151.  75. 141. 206. 135.]


# 3. Preparing the Data

The following code standardizes the features and splits the dataset into training and testing sets:

- **StandardScaler**: Standardizes the feature matrix by removing the mean and scaling to unit variance.
- **train_test_split**: Splits the dataset into training (80%) and testing (20%) sets.

The feature matrix **X** is transformed, and the dataset is divided into **X**, **X_test**, **y**, and **y_test**.


In [34]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

feature_scaler = StandardScaler()
target_scaler = StandardScaler()

X = feature_scaler.fit_transform(X)

# y = y.reshape(-1, 1)
# y = target_scaler.fit_transform(y)
# y = y.ravel()

X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Initializing Parameters

This code initializes the parameters for the linear regression model:

- **m, n**: The shape of the feature matrix **X**, where **m** is the number of samples (442) and **n** is the number of features (10).
- **w**: The weight vector, initialized to zeros with shape **[10,]**.
- **b**: The bias term, initialized to **0**.


In [35]:
m, n = X.shape   # m = 442, n = 10
w = np.zeros(n)  # Weight vector (shape: [10,])
b = 0            # Bias term (scalar)

# 5. Defining the Prediction Function

The prediction function is given by:

$$
\large \hat{y}_{(i)} = \sum_{j=1}^{n} w_j x_j^{(i)} + b
$$



**Where:**

 $$ \hat{y}_{(i)} = \text{predicted target value for the } i\text{-th sample.} $$
 $$ w_j = \text{weight for the } j\text{-th feature.} $$
 $$ x_j^{(i)} = \text{value of the } j\text{-th feature for the } i\text{-th sample.} $$

In [36]:
def predict(X, w, b):
    return np.dot(X, w) + b

This code defines the **predict** function, which calculates the predicted target values for each sample using the feature matrix **X**, weight vector **w**, and bias term **b**:

- **X**: The feature matrix (shape: [m, n]), where **m** is the number of samples and **n** is the number of features.
- **w**: The weight vector (shape: [n,]), which contains the learned weights for each feature.
- **b**: The bias term (scalar), which is added to the weighted sum.

The function uses the following formula to compute the predictions:

$$
\hat{y} = Xw + b
$$

# 6. Defining the Cost Function

The cost function measures how well the model's predictions match the actual target values and is defined as:

$$
\large J(w, b) = \frac{1}{2m} \sum_{i=1}^{m} \left( \hat{y}_{(i)} - y_{(i)} \right)^2
$$

Where **m** is the number of training examples.


In [37]:
def compute_cost(X, y, w, b):
    m = len(y)
    y_pred = predict(X, w, b)
    cost = (1 / (2 * m)) * np.sum((y_pred - y) ** 2)

    return cost

# 7. Computing the Gradients

Gradients are computed to update the weights and bias during training. The partial derivatives of the cost function with respect to each weight and the bias are:

$$
\large \frac{\partial J}{\partial w_j} = \frac{1}{m} \sum_{i=1}^{m} \left( \hat{y}_{(i)} - y_{(i)} \right) x_j^{(i)}
$$

.

$$
\large \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^{m} \left( \hat{y}_{(i)} - y_{(i)} \right)
$$

In [38]:
def compute_gradients(X, y, w, b):

    m = len(y)
    y_pred = predict(X, w, b)
    error = y_pred - y

    dw = (1 / m) * np.dot(X.T, error)
    db = (1 / m) * np.sum(error)

    return dw, db

# 8. Updating Parameters Using Gradient Descent

Parameters are updated using the learning rate and the gradients calculated:

$$
\large w := w - \alpha \frac{\partial J}{\partial w}
$$

$$
\large b := b - \alpha \frac{\partial J}{\partial b}
$$

**Where:**

$$
\alpha = \text{learning rate}
$$

In [39]:
def update_parameters(w, b, dw, db, learning_rate):

    w = w - learning_rate * dw
    b = b - learning_rate * db

    return w, b

# 9. Training the Model

This code performs the training of the linear regression model using gradient descent. It iteratively updates the model's parameters (weights and bias) and tracks the cost function:

- **learning_rate**: The step size for updating the parameters during each iteration. Set to **0.01**.
- **num_iterations**: The number of iterations for the gradient descent process. Set to **1000**.
- **cost_history**: A list that stores the cost value at each iteration to track the convergence of the model.

The training loop performs the following steps for each iteration:

1. **Prediction**: Uses the `predict` function to compute the predicted target values (**y_pred**).
2. **Cost Calculation**: Computes the cost (error) using the `compute_cost` function.
3. **Store Cost**: Appends the cost to the **cost_history** list for tracking.
4. **Gradient Calculation**: Computes the gradients for the weights and bias using `compute_gradients`.
5. **Parameter Update**: Updates the weights and bias using the `update_parameters` function with the calculated gradients and learning rate.
6. **Iteration Logging**: Every 100th iteration, prints the current iteration number and the corresponding cost.

In [65]:
w = np.zeros(n)
b = 0

learning_rate = .001
num_iterations = 10000
cost_history = []

parameters = {}

for i in range(num_iterations):

    y_pred = predict(X, w, b)
    cost = compute_cost(X, y, w, b)
    dw, db = compute_gradients(X, y, w, b)
    w, b = update_parameters(w, b, dw, db, learning_rate)

    if i % 1000 == 0:
      cost_history.append(cost)
      print(f'Iteration {i}: Cost = {cost:.4f}')

      parameters = {'weights': w.tolist(), 'bias': b}

Iteration 0: Cost = 14855.6615
Iteration 1000: Cost = 3043.6820
Iteration 2000: Cost = 1665.4576
Iteration 3000: Cost = 1479.7297
Iteration 4000: Cost = 1453.4973
Iteration 5000: Cost = 1449.3915
Iteration 6000: Cost = 1448.5068
Iteration 7000: Cost = 1448.1322
Iteration 8000: Cost = 1447.8521
Iteration 9000: Cost = 1447.5957


# 10. Evaluating Model Performance with Test Data


This code evaluates the performance of the trained linear regression model by calculating the final cost and Mean Squared Error (MSE), as well as printing the final learned parameters (weights and bias):

- **y_pred**: The predicted target values obtained from the `predict` function, using the final values of weights **w** and bias **b**.
- **final_cost**: The computed cost using the `compute_cost` function, representing the error between the predicted and actual values.
- **mse**: The Mean Squared Error, calculated as **2 times the final cost**, which is a common metric to evaluate regression models.


#### Key Metrics:

- **Residual Analysis**: The residuals are the differences between the actual and predicted values. This metric is calculated by taking the mean of the absolute differences between the actual test values (**y_test**) and predicted values (**y_pred_test**). A lower value indicates a better fit.
  
- **Mean Absolute Error (MAE)**: MAE is the average of the absolute differences between the actual and predicted values. It provides a simple measure of prediction accuracy.
  
- **Mean Squared Error (MSE)**: MSE is the average of the squared differences between the actual and predicted values. It penalizes larger errors more than MAE, making it sensitive to outliers.

- **Root Mean Squared Error (RMSE)**: RMSE is the square root of the MSE and provides an error metric in the same unit as the target variable, making it more interpretable.

- **R-squared (R²)**: R-squared measures the proportion of variance in the target variable that is predictable from the independent variables. A value closer to 1 indicates a better fit of the model to the data.

In [66]:
y_pred = predict(X, w, b)
final_cost = compute_cost(X, y, w, b)
mse = 2 * final_cost

print(f'Final Cost: {final_cost:.4f}')
print(f'Mean Squared Error: {mse:.4f}')
print('Final Weights:', w)
print('Final Bias:', b)

Final Cost: 1447.3486
Mean Squared Error: 2894.6971
Final Weights: [  2.00680987 -11.42232329  26.52055336  16.28157195  -6.5421879
  -4.79203362  -9.39340099   7.50916631  20.62575615   2.62898999]
Final Bias: 151.29598957844925


In [67]:
y_pred_test = predict(X_test, w, b)

# Calculate metrics

print(f"Residual Analysis : {(np.abs(y_test - y_pred_test)).mean()}")

mae = np.abs(y_test - y_pred_test).mean()
mse = ((y_test - y_pred_test)**2).mean()
rmse = np.sqrt(((y_test - y_pred_test)**2).sum()/len(y_test))


print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


SS_res = np.sum((y_test - y_pred_test)**2)        # Sum of squares of residuals(diff bet actual and predicted values)
SS_tot = np.sum((y_test - np.mean(y_test))**2)    # Total sum of squares

r2_ = 1 - (SS_res / SS_tot)

print(f"R-squared: {r2_:.4f}")

Residual Analysis : 42.902661677644126
Mean Absolute Error (MAE): 42.9027
Mean Squared Error (MSE): 2885.1498
Root Mean Squared Error (RMSE): 53.7136
R-squared: 0.4554


# 11. Conclusion


In this project, we successfully implemented a simple linear regression model from scratch using gradient descent. The following key steps were involved:

1. **Data Preparation**: We loaded and explored the dataset, performing necessary preprocessing, such as scaling features and splitting the data into training and test sets.
2. **Model Initialization**: The model's parameters (weights and bias) were initialized to zero.
3. **Prediction and Cost Calculation**: We defined a prediction function and a cost function to evaluate the accuracy of the model.
4. **Gradient Descent**: We calculated the gradients and applied gradient descent to update the model parameters iteratively.
5. **Model Training**: Over 1000 iterations, the model was trained, and the weights and bias were updated.
6. **Model Evaluation**: Finally, we evaluated the model's performance using cost and mean squared error (MSE), and printed the final learned parameters.

### Key Findings:
- The linear regression model was successfully trained and optimized using gradient descent.
- The final learned weights and bias were printed, showing the results of the optimization.
- The model's performance was assessed using the cost and MSE, providing a clear understanding of its predictive ability.

### Benefits of Linear Regression:
- **Simplicity and Interpretability**: Linear regression is easy to implement and understand, making it a great starting point for many machine learning problems.
- **Efficiency**: The model is computationally inexpensive and can handle large datasets effectively.
- **Well-Suited for Linearly-Related Data**: It is ideal for predicting continuous target variables that have a linear relationship with the features.

### Drawbacks of Linear Regression:
- **Assumption of Linearity**: Linear regression assumes a linear relationship between the features and target variable, which may not always hold true.
- **Sensitivity to Outliers**: Outliers can have a significant impact on the model's performance, as linear regression is sensitive to extreme values.
- **Multicollinearity**: High correlation between input features can lead to unreliable parameter estimates and degrade the model's performance.

In conclusion, while linear regression provides an effective and interpretable model for problems with a linear relationship, its limitations in handling non-linearity and outliers must be considered. For more complex relationships, other models such as polynomial regression or more advanced machine learning algorithms may be necessary.

---

# 12. Comparison with Sklearn Linear Regression

In this section, we use **scikit-learn** to train a linear regression model and evaluate its performance using the test data. The model is trained on the features (**X**) and target values (**y**) and then evaluated using **Mean Squared Error (MSE)** and **R-squared (R²)** metrics.

### Code Explanation:

1. **Model Initialization**:
   - `model = LinearRegression()`: Initializes a linear regression model using **scikit-learn**'s `LinearRegression` class.

2. **Model Training**:
   - `model.fit(X, y)`: Fits the model to the training data (**X**, **y**) by finding the optimal values for the weights and bias.

3. **Prediction**:
   - `y_pred_test = model.predict(X_test)`: Uses the trained model to predict the target values (**y_pred_test**) for the test data (**X_test**).

4. **Model Evaluation**:
   - `mse = mean_squared_error(y_test, y_pred_test)`: Computes the **Mean Squared Error (MSE)** between the actual target values (**y_test**) and the predicted values (**y_pred_test**). MSE measures the average squared difference between the predicted and actual values.
   - `r2 = r2_score(y_test, y_pred_test)`: Calculates the **R-squared (R²)** score, which indicates the proportion of the variance in the target variable that is explained by the model.

5. **Print Results**:
   - The code prints the computed **MSE** and **R-squared** values, providing an understanding of how well the model fits the data.

In [70]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

model = SGDRegressor(
    loss='squared_error',
    alpha=0.0,
    learning_rate='constant',  # Set to 'constant' for a fixed learning rate
    eta0=0.01,                 # Initial learning rate (will remain fixed)
    max_iter=1000,
    random_state=42
    # tol = None                    # no early stopping
  )


model.fit(X, y)  # Use training data for fitting

# 4. Make predictions on the test set
y_pred_test = model.predict(X_test)

# 5. Evaluate the model
mse = mean_squared_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R-squared: {r2:.4f}")

Mean Squared Error: 2900.1936
R-squared: 0.4526


In [ ]:
print('scratch model weights : ' , model.coef_)
print('sklearn model weights : ' , w)

# Bonus Model Training

In [71]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


np.random.seed(42)
X = np.random.rand(100, 1) * 10
y = 2 * X + 5 + np.random.randn(100, 1) * 2

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R-squared: {r2:.4f}")

Mean Squared Error: 2.6148
R-squared: 0.9287
